<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, EdX<br>
            Dr. Eric Van Dusen <br>
        Akhil Venkatesh <br>
</table>

# Lecture 4.4: Macroeconomics Laws

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datascience import * 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.optimize import curve_fit
import math
from math import *
import seaborn as sns
import itertools
%matplotlib inline

### Let's Start by reading in the Data from the previous lecture Notebook


In [ ]:
macroeconomics=pd.read_csv('macroeconomics.csv')
macroeconomics.head()

In [ ]:
potential_gdp=pd.read_csv('potential_gdp.csv')
potential_gdp

## Phillips Curve
Phillips Curve describes a negative relationship between inflation and unemployment. 

$$ \pi = - f(u)$$

where $\pi$ is the inflation rate, $u$ is the unemployment rate. 

**Let's first create a dataframe with relevant data, i.e. inflation rate and unemployment rate.**

In [ ]:
phillips_curve = macroeconomics[["DATE", "UNRATE_lin", "CPILFESL_pc1", "NROU_lin"]].copy()
phillips_curve["decade"] = phillips_curve["DATE"].apply(lambda date: int(date[:4]) // 10 * 10)
phillips_curve.set_index("DATE", inplace=True)
phillips_curve = phillips_curve.apply(pd.to_numeric)
phillips_curve.head()

Friendly note: you should not use this piece of code for your lab, because the dataframe is slightly different!

### Empirical Relationship

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(x=phillips_curve["UNRATE_lin"], y=phillips_curve["CPILFESL_pc1"])
plt.xlabel("Unemployment")
plt.ylabel("Core Inflation")
plt.show()

**Now we will try a fit a curve to the empirical data of unemployment and core inflation.** 

In [ ]:
unique_vals = pd.DataFrame([[k,v.values] for k,v in phillips_curve.groupby('UNRATE_lin').groups.items()], 
                      columns=['UNRATE_lin','indices'])
unique_vals["indices"] = unique_vals["indices"].apply(lambda item: item[0])
unique_vals["CPILFESL_pc1"] = unique_vals["indices"].apply(lambda item: phillips_curve.loc[item, "CPILFESL_pc1"])
unique_vals["NROU_lin"] = unique_vals["indices"].apply(lambda item: phillips_curve.loc[item, "NROU_lin"])
unique_vals["decade"] = unique_vals["indices"].apply(lambda item: phillips_curve.loc[item, "decade"])
unique_vals.set_index("indices", inplace=True)
unique_vals.head()

In [ ]:
def aesthetic_pc(x, a):
    return a / x

In [ ]:
popt, _ = curve_fit(aesthetic_pc, unique_vals["UNRATE_lin"], unique_vals["CPILFESL_pc1"])

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(unique_vals["UNRATE_lin"], aesthetic_pc(unique_vals["UNRATE_lin"], *popt), color="red")
plt.scatter(x=phillips_curve["UNRATE_lin"], y=phillips_curve["CPILFESL_pc1"])
plt.xlabel("Unemployment")
plt.ylabel("Core Inflation")
plt.title("Phillips Curve", fontsize=16)
plt.show()

### Empirical Relationship Overtime
How does the empirical Phillips Curve change over time?

In [ ]:
plt.figure(figsize=(15, 10))
sns.scatterplot(x="UNRATE_lin", y="CPILFESL_pc1", data=phillips_curve, 
                hue="decade", palette="bright", alpha=0.75)
plt.xlabel("Unemployment")
plt.ylabel("Core Inflation")
plt.show()

To show this more clearly, try toggling the widgets!

In [ ]:
def plot_subset(start, length):
    start_str = str(start) + "-01-01"
    end_str = str(start + length) + "-01-01"
    
    subset = phillips_curve[(phillips_curve.index >= start_str) & (phillips_curve.index <= end_str)]
    
    plt.figure(figsize=(12, 8))
    plt.scatter(subset["UNRATE_lin"], subset["CPILFESL_pc1"])
    plt.xlim(3, 10)
    plt.ylim(0, 12)
    plt.xlabel("Unemployment Rate", fontsize=16)
    plt.ylabel("Inflation Rate", fontsize=16)
    plt.title(f"Time Period: {start} - {start + length}", fontsize=16)

l = widgets.RadioButtons(
    options=[5, 10, 20],
    value=5,
    description='Length:',
    disabled=False
)
s = widgets.IntSlider(min=1960, max=2020-l.value, step=2)

interact(plot_subset, start=s, length=l);

<div class="alert alert-info">
<b> What do you notice about the relationship between unemployment and inflation over time?
</div>

---